In [15]:
import logging
import coloredlogs
import requests
from time import sleep

In [21]:
logging.basicConfig()
logger = logging.getLogger("poll-ooni")
# logger.setLevel(logging.DEBUG)
coloredlogs.install()
coloredlogs.install(level='DEBUG')
# coloredlogs.install(level='INFO')

In [17]:
config = {
    "sleep-times": {
        "ooni-poll": 60*5,
        "ooni-paginate": 2,
    }
}


# Getting the data we need

## Measuremnets from OONI

In [39]:
def api_query (query: str, results=[], queries=1, max_queries=None) -> list:
    '''Recursively query the API, up to `max_queries`. (If `max_queries=None`, we
    will paginate through the results as long as they run).
    '''
    base_url = 'https://api.ooni.io/api/v1/'
    query = '{!s}{!s}'.format(base_url, query)
    try:
        resp =  requests.get(query).json()
        results = results + resp['results']
        next_url = resp['metadata']['next_url']
        if max_queries is not None and queries > max_queries:
            return results
        if next_url:
            # sleep so as to not overwhelm the endpoint
            sleep(config['sleep-times']['ooni-paginate'])
            # remove base url to perfrom the query
            next_url = next_url.split('api/v1')[1]
            return api_query(next_url, results, queries+1, max_queries)
        return results
    except Exception as inst:
        # if we have an error,
        logger.warning("Error querying API: {!s}".format(inst))
        # just return what we've collected
        # (at worst, `results` will be `[]`)
        return results


In [40]:
def query_recent_measurements (max_queries=5) -> list:
    '''Queries all recent measurements, up to specified maximum number of queries.'''
    return api_query('measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000', max_queries=max_queries)

# def query_measurements_after (time) -> list:
#     '''Queries all measurements after time.'''
#     return api_query('measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since={!s}'.format(time))

# def get_measurement_time (measurement) -> str:
#     '''Returns a time format that can be queried'''
#     return measurement['measurement_start_time'][:-1]

# def most_recent_measurement_time (measurements) -> str:
#     return get_measurement_time(measurements[0])

def get_blocking_type (measurement) -> str:
    '''Get blocking type, if available.'''
    try:
        return measurement['scores']['analysis']['blocking_type']
    except:
        return None

In [42]:
measurements = query_recent_measurements()

2021-05-24 11:13:18 congratulations urllib3.connectionpool[30919] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-05-24 11:13:19 congratulations urllib3.connectionpool[30919] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000 HTTP/1.1" 200 835514
2021-05-24 11:13:22 congratulations urllib3.connectionpool[30919] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-05-24 11:13:23 congratulations urllib3.connectionpool[30919] DEBUG https://api.ooni.io:443 "GET /api/v1//measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&offset=1000 HTTP/1.1" 404 None
2021-05-24 11:13:23 congratulations poll-ooni[30919] WARNING Error querying API: Expecting value: line 1 column 1 (char 0)


In [137]:
m = measurements[505]
m

{'anomaly': True,
 'confirmed': False,
 'failure': False,
 'input': 'http://www.newnownext.com/franchise/the-backlot/',
 'measurement_start_time': '2021-05-24T16:56:23Z',
 'measurement_url': 'https://ams-pg.ooni.org/api/v1/raw_measurement?report_id=20210524T153727Z_webconnectivity_BR_271354_n1_v6EcCDifm3IJDoaO&input=http%3A%2F%2Fwww.newnownext.com%2Ffranchise%2Fthe-backlot%2F',
 'probe_asn': 'AS271354',
 'probe_cc': 'BR',
 'report_id': '20210524T153727Z_webconnectivity_BR_271354_n1_v6EcCDifm3IJDoaO',
 'scores': {'analysis': {'blocking_type': 'dns'},
  'blocking_country': 0.0,
  'blocking_general': 1.0,
  'blocking_global': 0.0,
  'blocking_isp': 0.0,
  'blocking_local': 0.0},
 'test_name': 'web_connectivity'}

In [143]:
m_detail = requests.get(m['measurement_url']).json()

m_detail['test_keys']['queries']
m_detail

2021-05-24 12:09:02 congratulations urllib3.connectionpool[30919] DEBUG Starting new HTTPS connection (1): ams-pg.ooni.org:443
2021-05-24 12:09:02 congratulations urllib3.connectionpool[30919] DEBUG https://ams-pg.ooni.org:443 "GET /api/v1/raw_measurement?report_id=20210524T153727Z_webconnectivity_BR_271354_n1_v6EcCDifm3IJDoaO&input=http%3A%2F%2Fwww.newnownext.com%2Ffranchise%2Fthe-backlot%2F HTTP/1.1" 200 129010


{'annotations': {'engine_name': 'ooniprobe-engine',
  'engine_version': '3.9.1',
  'platform': 'linux'},
 'data_format_version': '0.2.0',
 'input': 'http://www.newnownext.com/franchise/the-backlot/',
 'measurement_start_time': '2021-05-24 16:56:23',
 'probe_asn': 'AS271354',
 'probe_cc': 'BR',
 'probe_ip': '127.0.0.1',
 'probe_network_name': '<unknown>',
 'report_id': '20210524T153727Z_webconnectivity_BR_271354_n1_v6EcCDifm3IJDoaO',
 'resolver_asn': 'AS271354',
 'resolver_ip': '150.164.0.80',
 'resolver_network_name': '<unknown>',
 'software_name': 'ooniprobe-cli',
 'software_version': '3.9.1',
 'test_helpers': {'backend': {'address': 'https://wcth.ooni.io',
   'type': 'https'}},
 'test_keys': {'agent': 'redirect',
  'client_resolver': '150.164.0.80',
  'retries': None,
  'socksproxy': None,
  'network_events': [{'address': '200.143.247.42:80',
    'failure': None,
    'operation': 'connect',
    'proto': 'tcp',
    't': 2.219990817,
    'tags': ['tcptls_experiment']},
   {'address': '

## Get IP from URL

In [89]:
inputs = [m['input'] for m in measurements]

In [213]:
import socket
import urllib.parse

def get_hostname (url):
    return urllib.parse.urlparse(url).netloc

def get_ip (url: str) ->  Optional[str]:
    hostname = get_hostname(url)
    try:
        return socket.gethostbyname(hostname)
    except Exception as inst:
            logger.warning(f"Error looking up IP of hostname {hostname}: {inst}")
            return None


In [174]:
my_ip = get_ip(inputs[505])
my_ip

'157.131.218.41'

## Get geolocation from IP

In [211]:
import geoip2.database
from typing import Optional

def ip_to_alpha2 (ip: str) -> Optional[str]:
    with geoip2.database.Reader('dbip-country-lite-2021-05.mmdb') as reader:
        try:
            response = reader.country(ip)
            return response.country.iso_code
        except Exception as inst:
            # if we have an error,
            logger.warning(f"Error looking up country code of IP {ip}: {inst}")
            return None
    
get_alpha2(my_ip)

'US'

## Putting it all together

In [207]:
def url_to_alpha2 (url):
    return ip_to_alpha2(get_ip(url))

def measurement_to_alpha2 (measurement):
    url = measurement['input']
    return url_to_alpha2(url)

In [214]:
for m in measurements:
    url = m['input']
    print(url, end=': ')
    alpha2 = url_to_alpha2(url)
    print(alpha2)


https://www.cna.com.tw/: TW
https://nordvpn.com/: US
https://www.change.org/: US
http://www.tiananmenmother.org/: US
http://www.ftchinese.com/: US
https://www.ndi.org/: US
http://www.hrea.org/: US
https://blog.mozilla.org/: US
http://www.ifeminists.com/: US
http://ocsp.int-x3.letsencrypt.org/: US
https://www.bbc.com/burmese/: CA
http://www.theepochtimes.com/: US
https://www.brookings.edu/center/john-l-thornton-china-center/: US
https://fedoramagazine.org/: US
https://doubleclick.net/: US
http://bancariosclassistas.blogspot.com/: CA
http://doubleclick.net/: US
https://weblog.savetibet.org/: US
http://www.animalliberationfront.com/: US
https://bridges.torproject.org/: DE
http://www.zensur.freerk.com/: DE
https://bisexual.org/: US
http://de.lirio.us/: NL
http://www.dailymail.co.uk/: US
http://peacefire.org/: US
http://juntosomos-fortes.blogspot.com/: CA
http://www.goarch.org/: US
https://git.io/: US
http://guampanews.blogspot.com/: CA
http://polentanews.blogspot.com/: CA
http://www.askmos

2021-05-24 13:20:05 congratulations poll-ooni[30919] WARNING Error looking up country code of IP 127.0.0.1: The address 127.0.0.1 is not in the database.
2021-05-24 13:20:05 congratulations poll-ooni[30919] WARNING Error looking up country code of IP 127.0.0.1: The address 127.0.0.1 is not in the database.


None
https://upload.twitter.com/robots.txt: US
http://polentanews.blogspot.com/: CA
https://www.reuters.com/: US
http://www.ifeminists.com/: US
http://www.iwantim.com/: None
http://pravoslavie.ru/: RU
http://bluesystem.info/: GB
https://chinadigitaltimes.net/: US
http://www.irna.ir/: IR
http://www.lesbi.ru/: RU
http://folha.arcauniversal.com.br/: BR
https://www.hidemyass.com/: US
https://freetibet.org/: GB
http://www.newnownext.com/franchise/the-backlot/: US
http://www.kurtuluscephesi.com/: US
http://lurkmore.to/: LU
http://anonymouse.org/: DE
http://anon.inf.tu-dresden.de/: DE
http://juntosomos-fortes.blogspot.com/: CA
http://guampanews.blogspot.com/: CA
https://hkoutlanders.com/: US
http://pravoslavie.ru/: RU
http://bancariosclassistas.blogspot.com/: CA
https://tibetaction.net/: US
https://www.getoutline.org/: CA
https://livemy.info/: BY
https://foreignpolicy.com/author/david-bosco/: US
https://www.reuters.com/: US
https://www.jw.org/: US
https://www.upi.com/Top_News/World-News/2012/

2021-05-24 13:20:08 congratulations poll-ooni[30919] WARNING Error looking up country code of IP 127.0.0.1: The address 127.0.0.1 is not in the database.


None
https://wqw2010.blogspot.com/: CA
http://www.judaism.com/: US
https://global.blackberry.com/en/home.html: US
https://www.hrichina.org/chs/zhong-guo-ren-quan-shuang-zhou-kan/: US
http://th.wikipedia.org/wiki/%E0%B8%9E%E0%B8%A3%E0%B8%B0%E0%B8%9A%E0%B8%B2%E0%B8%97%E0%B8%AA%E0%B8%A1%E0%B9%80%E0%B8%94%E0%B9%87%E0%B8%88%E0%B8%9E%E0%B8%A3%E0%B8%B0%E0%B8%9B%E0%B8%A3%E0%B8%A1%E0%B8%B4%E0%B8%99%E0%B8%97%E0%B8%A3%E0%B8%A1%E0%B8%AB%E0%B8%B2%E0%B8%A0%E0%B8%B9%E0%B8%A1%E0%B8%B4%E0%B8%9E%E0%B8%A5%E0%B8%AD%E0%B8%94%E0%B8%B8%E0%B8%A5%E0%B8%A2%E0%B9%80%E0%B8%94%E0%B8%8A: US
http://zaa.xq28.org/: BR
https://www.hotspotshield.com/: US
http://www.hotspotshield.com/: US
https://www.sbs.com.au/language/coronavirus?cid=infocus: US
https://twitter.com/aiww/: US
https://mail.yahoo.com/: US
http://www.linkedin.com/: US
https://protonvpn.com/: CH
https://qsbr.fs.quoracdn.net/-4-web.entry.js.out-34-8f7d03da0b7a24ae.webpack: CA
http://th.wikipedia.org/wiki/%E0%B8%9E%E0%B8%A3%E0%B8%B0%E0%B8%9A%E0%B8%B2%E0%B8%97

...

2021-05-24 13:21:29 congratulations poll-ooni[30919] WARNING Error looking up country code of IP 127.0.0.1: The address 127.0.0.1 is not in the database.


BR
http://prompteetsincere.blogspot.com/: CA
http://blogdaunr.blogspot.com/: CA
http://www.javjunkies.com/: US
http://donbass.ua/: US
http://coisasdesp.blogspot.com/: CA
https://boingboing.net/: US
http://www.zensur.freerk.com/: DE
http://www.iwantim.com/: None
http://www.zensur.freerk.com/: DE
http://www.cannaweed.com/: US
http://renatamielli.blogspot.com/: CA
http://drugs-forum.com/: US
http://www.venus.com/: US
http://blogdoronald.blogspot.com/: CA
http://sanatoriodanoticia.blogspot.com/: CA
http://www.hanes.com/: CA
http://www.law-lib.utoronto.ca/Diana/: CA
http://anselmocarvalho.blogspot.com/: CA
http://liberdadedeexpressaovirtual.blogspot.com/: CA
http://khilafah.net/: DE
http://imrussia.org/en/: US
http://estoudeolho.blogspot.com/: CA
http://omapadobrasil.blogspot.com/: CA
https://boingboing.net/: US
http://www.resistance88.com/: US
https://ultrasurf.us/: US
http://www.888casino.com/: US
http://www.casinotropez.com/: US
http://www.kerch.com/: US
http://www.hanes.com/: CA
http://

2021-05-24 13:21:30 congratulations poll-ooni[30919] WARNING Error looking up country code of IP 127.0.0.1: The address 127.0.0.1 is not in the database.
2021-05-24 13:21:30 congratulations poll-ooni[30919] WARNING Error looking up country code of IP 127.0.0.1: The address 127.0.0.1 is not in the database.


None
http://www.iwantim.com/: None
http://www.1gay.ru/: US
https://www.opensocietyfoundations.org/: US
https://mail.protonmail.com/: CH
http://www.hizb-ut-tahrir.org/: DE
https://www.usrf.us/: US
http://gaytoday.com/: US
http://www.marcelinovieira.rn.gov.br/: 

KeyboardInterrupt: 

## Get TLD jurisdiction


In [224]:
from src.w3techs.utils import get_country
from tldextract import extract

In [243]:
def get_tld_jurisdiction (url: str) -> str:
    '''
    Takes a URL and gets an Alpha 2
    representing the jurisdiction of the URL's top-level domain.
    '''
    tld = extract(url)
    # get last item in url
    # e.g., '.com.br' should be '.br'
    tld = tld.suffix
    tld = tld.split('.')[-1]
    # put it
    tld_str = f'.{tld}'
#     return tld_str
    return get_country(tld_str)

get_tld_jurisdiction('mycool.com.br')

'BR'

In [244]:
for m in measurements:
    url = m['input']
    print(url, end=' - ')
    print(get_tld_jurisdiction(url))

2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .to
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .is
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .mil
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .fm
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .website
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .to
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .name
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .is
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .int
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .is
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .


https://www.cna.com.tw/ - TW
https://nordvpn.com/ - US
https://www.change.org/ - US
http://www.tiananmenmother.org/ - US
http://www.ftchinese.com/ - US
https://www.ndi.org/ - US
http://www.hrea.org/ - US
https://blog.mozilla.org/ - US
http://www.ifeminists.com/ - US
http://ocsp.int-x3.letsencrypt.org/ - US
https://www.bbc.com/burmese/ - US
http://www.theepochtimes.com/ - US
https://www.brookings.edu/center/john-l-thornton-china-center/ - US
https://fedoramagazine.org/ - US
https://doubleclick.net/ - US
http://bancariosclassistas.blogspot.com/ - US
http://doubleclick.net/ - US
https://weblog.savetibet.org/ - US
http://www.animalliberationfront.com/ - US
https://bridges.torproject.org/ - US
http://www.zensur.freerk.com/ - US
https://bisexual.org/ - US
http://de.lirio.us/ - US
http://www.dailymail.co.uk/ - GB
http://peacefire.org/ - US
http://juntosomos-fortes.blogspot.com/ - US
http://www.goarch.org/ - US
https://git.io/ - US
http://guampanews.blogspot.com/ - US
http://polentanews.blogsp

2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .mil
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .news
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .foundation
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .agency
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .news
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .gov
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .news
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .host
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .cat
2021-05-24 13:35:19 congratulations root[30919] INFO Cannot find country for .news
2021-05

 - None
http://www.becteroradio.com/ - US
http://ipvnews.org/ - US
https://tx.me/ - US
https://ichkeria.info/ - US
https://maidan.org.ua/ - UA
http://egoestagiario.com/ - US
https://myabortionstory.tumblr.com/ - US
https://www.segodnya.ua/ - UA
https://vortex-team.org/ - US
http://www.agazeta-acre.com.br/ - BR
https://15minut.org/ - US
http://www.newnownext.com/ - US
http://www.quantico.marines.mil/ - None
http://www.kasparov.ru/ - RU
http://advocacy.globalvoicesonline.org/ - US
https://web.archive.org/ - US
http://gaytoday.com/ - US
http://www.hrcr.org/ - US
http://www.tribunadenobres.com.br/ - BR
http://www.lawaraujo.blogspot.com/ - US
http://www.themwl.org/ - US
https://www.opensocietyfoundations.org/ - US
http://www.iwantim.com/ - US
https://www.anonymizer.com/ - US
https://amnesty.org.ru/ - RU
https://glavcom.ua/ - UA
http://www.khilafah.com/ - US
http://polentanews.blogspot.com/ - US
https://www.hidemyass.com/ - US
http://naoabandoneseumelhoramigo.blogspot.com/ - US
https://torre

# Model the datatype

In [220]:
get_ip('http://www.exmormon.org/')

'204.238.168.42'